In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("nba_games1.csv", index_col=0)

In [5]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [6]:
df = df.sort_values("date")
df = df.reset_index(drop=True)

In [7]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [8]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

In [9]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1618397414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [10]:
df["target"] = df["target"].astype(int, errors="ignore")

In [11]:
df["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [12]:
nulls = pd.isnull(df)

In [13]:
nulls = nulls.sum()

In [14]:
nulls = nulls[nulls > 0]

In [15]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [16]:
df = df[valid_columns].copy()

In [17]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [18]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [19]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [21]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.277279,0.554502,0.317647,GSW,0.451923,1.0,2016,2015-10-27,False,0
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.160462,0.345972,0.317647,CHI,0.317308,1.0,2016,2015-10-27,False,1
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.088575,0.232227,0.329412,CLE,0.298077,0.0,2016,2015-10-27,True,1
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.215661,0.530806,0.505882,NOP,0.298077,0.0,2016,2015-10-27,True,1
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.019255,0.203791,0.317647,DET,0.403846,0.0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.182285,0.208531,0.411765,GSW,0.413462,0.0,2022,2022-06-10,False,0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.928113,1.000000,0.411765,BOS,0.288462,0.0,2022,2022-06-13,True,1
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.181001,0.630332,0.352941,GSW,0.384615,1.0,2022,2022-06-13,False,0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.120668,0.459716,0.400000,GSW,0.375000,0.0,2022,2022-06-16,False,2


In [22]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [23]:
predictors = list(selected_columns[sfs.get_support()])

In [24]:
predictors

['mp',
 'fg%',
 '3p%',
 'orb',
 'ts%',
 'usg%',
 '3p%_max',
 'ft_max',
 'fta_max',
 '+/-_max',
 'drb%_max',
 'trb%_max',
 'tov%_max',
 'usg%_max',
 'mp_opp',
 'fg_opp',
 '3p_opp',
 'ft%_opp',
 'blk_opp',
 'usg%_opp',
 'fga_max_opp',
 '3p_max_opp',
 'ft_max_opp',
 'ft%_max_opp',
 'blk_max_opp',
 'pf_max_opp',
 'pts_max_opp',
 'drb%_max_opp',
 'blk%_max_opp',
 'usg%_max_opp']

In [25]:
def backtest(data, model, predictors, start=2, step=1):
    # List of dataframes, each df is the prediction for a single season
    all_predictions = []
    
    # All the seasons that exist in the data
    seasons = sorted(df["season"].unique())
    
    # Loop through the seasons
    # For every new season, take all the previous seasons as training data
    for i in range(start, len(seasons), step):
        # Season Number
        season = seasons[i]
        
        # training data, all data before current season
        train = data[data["season"] < season]
        # testing data, used to generate predictions for current season
        test = data[data["season"] == season]

        # Takes training data and figures out the best predictors for the target 
        model.fit(train[predictors], train["target"])

        # Makes predictions on the test set & convert to Pandas series
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        # Turn into a combined df
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)


In [26]:
predictions = backtest(df, rr, predictors)
predictions

,actual,prediction
5250,1,1
5251,1,1
5252,0,0
5253,1,0
5254,0,1
...,...,...
17767,0,0
17768,1,1
17769,0,1
17770,2,1


In [27]:
# Creates one metric that tells how accurate our model is 
from sklearn.metrics import accuracy_score

# Gives us percentage correct
predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["prediction"])

0.5485110470701249

In [28]:
# Splits the df in 2 groups (home & away) & finds their winning percentage
df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

home
0.0    0.428314
1.0    0.571686
dtype: float64

In [29]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

import numpy as np

def find_team_averages(team):
    # Select only numeric columns
    team_numeric = team.select_dtypes(include=[np.number])
    
    # Apply rolling mean with a window size of 10
    rolling = team_numeric.rolling(10).mean()
    
    # Optionally, if you want to keep the original non-numeric columns as well:
    result = team.copy()  # Create a copy to avoid modifying the original DataFrame
    result[team_numeric.columns] = rolling  # Update only the numeric columns with rolling mean
    
    return result

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

In [30]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NOP,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,CLE,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,CHI,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,GSW,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ATL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.381818,0.292647,0.428230,0.468966,0.477273,0.448100,0.434884,0.373016,0.764177,...,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,False,BOS,2022.0
17768,0.0,0.502273,0.364706,0.517703,0.455172,0.481818,0.440736,0.320930,0.282540,0.757993,...,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,True,GSW,2022.0
17769,0.0,0.354545,0.279412,0.404545,0.437931,0.465152,0.429572,0.434884,0.385714,0.736639,...,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,False,BOS,2022.0
17770,0.0,0.354545,0.294118,0.389952,0.434483,0.459091,0.431710,0.406977,0.357143,0.754142,...,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,False,BOS,2022.0


In [32]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NOP,NaN
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,CLE,NaN
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,CHI,NaN
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,GSW,NaN
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ATL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,False,BOS,2022.0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,True,GSW,2022.0
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,False,BOS,2022.0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,False,BOS,2022.0


In [33]:
df = df.dropna()
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
243,0.0,0.522727,0.382353,0.523923,0.344828,0.333333,0.457245,0.255814,0.238095,0.708285,...,0.0679,0.413522,0.124134,0.361611,0.449412,0.347115,0.4,True,ATL,2016.0
251,0.0,0.659091,0.426471,0.645933,0.620690,0.515152,0.562945,0.325581,0.238095,0.927655,...,0.0772,0.469497,0.219641,0.394787,0.531765,0.324038,0.5,True,GSW,2016.0
252,0.0,0.386364,0.382353,0.358852,0.206897,0.181818,0.445368,0.511628,0.412698,0.827305,...,0.1145,0.437841,0.138126,0.507109,0.360000,0.351923,0.6,True,MEM,2016.0
253,0.0,0.500000,0.382353,0.497608,0.344828,0.318182,0.475059,0.325581,0.349206,0.593932,...,0.0759,0.512159,0.133633,0.277251,0.388235,0.308654,0.4,True,IND,2016.0
256,0.0,0.318182,0.132353,0.500000,0.275862,0.272727,0.432304,0.581395,0.444444,0.879813,...,0.0982,0.313312,0.179974,0.500000,0.471765,0.380769,0.5,False,POR,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,False,BOS,2022.0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,True,GSW,2022.0
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,False,BOS,2022.0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,False,BOS,2022.0


In [35]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1688004556.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["home_next"] = add_col(df, "home")
/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1688004556.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["team_opp_next"] = add_col(df, "team_opp")
/var/folders/_p/64h5xr191936rbqt54d0mfww0000gn/T/ipykernel_2738/1688004556.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [37]:
df = df.copy()

In [38]:
full = df.merge(
    df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
    left_on=["team", "date_next"],
    right_on=["team_opp_next", "date_next"]
)


In [39]:
full

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.00,0.477273,0.500000,0.375598,0.379310,0.348485,0.483373,0.441860,0.396825,0.730455,...,0.273427,0.270616,0.478824,0.308654,0.6,True,TOR,2016.0,SAC,TOR
1,0.00,0.340909,0.250000,0.413876,0.310345,0.257576,0.509501,0.511628,0.412698,0.827305,...,0.124904,0.404739,0.408235,0.428846,0.2,True,SAC,2016.0,TOR,SAC
2,0.50,0.409091,0.455882,0.330144,0.482759,0.515152,0.437055,0.372093,0.412698,0.568261,...,0.153273,0.344076,0.384706,0.319231,0.7,False,DET,2016.0,CLE,DET
3,0.25,0.545455,0.544118,0.416268,0.413793,0.454545,0.419240,0.186047,0.142857,0.883314,...,0.276508,0.352607,0.482353,0.316346,0.7,False,TOR,2016.0,GSW,TOR
4,0.00,0.340909,0.558824,0.186603,0.206897,0.469697,0.203088,0.139535,0.111111,0.854142,...,0.156739,0.470142,0.391765,0.436538,0.6,False,NOP,2016.0,DEN,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15769,0.00,0.545455,0.426471,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.235173,0.562085,0.552941,0.429808,0.4,False,GSW,2022.0,BOS,GSW
15770,0.00,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.170603,0.431754,0.522353,0.348077,0.5,False,BOS,2022.0,GSW,BOS
15771,0.00,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.242875,0.567773,0.575294,0.394231,0.4,True,GSW,2022.0,BOS,GSW
15772,0.00,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.174711,0.438863,0.483529,0.350000,0.5,False,BOS,2022.0,GSW,BOS


In [41]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAC,TOR,TOR,SAC,2015-11-15
1,TOR,SAC,SAC,TOR,2015-11-15
2,CLE,DET,DET,CLE,2015-11-17
3,GSW,TOR,TOR,GSW,2015-11-17
4,DEN,NOP,NOP,DEN,2015-11-17
...,...,...,...,...,...
15769,BOS,GSW,GSW,BOS,2022-06-10
15770,GSW,BOS,BOS,GSW,2022-06-13
15771,BOS,GSW,GSW,BOS,2022-06-13
15772,GSW,BOS,BOS,GSW,2022-06-16


In [42]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [43]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [44]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [45]:
predictors = list(selected_columns[sfs.get_support()])

In [46]:
predictors

['3p%',
 'usg%',
 'fta_max',
 'drb_max',
 'ts%_max',
 '3p_opp',
 'trb_opp',
 'blk%_opp',
 'usg%_opp',
 'fg_max_opp',
 'ft_max_opp',
 'ft%_max_opp',
 'pts_10_x',
 'usg%_10_x',
 '+/-_max_10_x',
 'total_10_x',
 'ft%_opp_10_x',
 'pf_opp_10_x',
 'usg%_opp_10_x',
 'efg%_max_opp_10_x',
 'drtg_max_opp_10_x',
 'home_next',
 'usg%_10_y',
 'ortg_10_y',
 '+/-_max_10_y',
 'ft%_opp_10_y',
 'usg%_opp_10_y',
 'fg%_max_opp_10_y',
 'ast%_max_opp_10_y',
 'drtg_max_opp_10_y']

In [47]:
predictions = backtest(full, rr, predictors)

In [48]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.6316121474272326